In [1]:
#!pip3 install pandas
#!pip3 install PyArrow
from pyspark.sql.functions import col

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

In [3]:
import os
import time
import json
import requests
import xml.etree.ElementTree as ET
import datetime
import subprocess

In [4]:
if os.path.exists("/etc/hadoop/conf/hive-site.xml"):
    tree = ET.parse("/etc/hadoop/conf/hive-site.xml")
    root = tree.getroot()
    for prop in root.findall("property"):
        if prop.find("name").text == "hive.metastore.warehouse.dir":
            storage = (
                prop.find("value").text.split("/")[0]
                + "//"
                + prop.find("value").text.split("/")[2]
            )

In [5]:
os.environ["STORAGE"] = storage

In [6]:
print(storage)

s3a://go01-demo


In [7]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
    .config("spark.rpc.message.maxSize", "1024")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .getOrCreate()
 #   .config("spark.driver.cores", 4)\
 #   .config("spark.driver.memory", "8g")\
    

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [8]:
customers_df = spark.read.csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data', header=True)

In [9]:
#customers_df.select("customer_id").show()

In [10]:
!pip3 install faker

In [11]:
customers_df.dtypes

[('customer_id', 'string'),
 ('username', 'string'),
 ('name', 'string'),
 ('gender', 'string'),
 ('email', 'string'),
 ('occupation', 'string'),
 ('birthdate', 'string'),
 ('address', 'string'),
 ('device_id', 'string')]

In [12]:
max_current_cust_id = int(customers_df.select(F.max("customer_id")).collect()[0]['max(customer_id)'])

In [13]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [14]:
from faker import Faker
fake = Faker(seed=3)

In [15]:
!pip3 install pandas
import pandas as pd

In [16]:
import random
import numpy as np

In [19]:
#don't run again

In [37]:
#ct_hist_df = spark.read.option("header","true").parquet("s3a://go01-demo/cde-workshop/clickthrough/historical")

In [86]:
#hist_ded_ids = ct_hist_df.select("device_id").sample(.01).toPandas()

In [87]:
#unique_arr = np.unique(hist_ded_ids.device_id)

In [88]:
#unique_arr

array(['000032d7', '000070cc', '00038618', ..., 'ffff60f9', 'ffff9249',
       'ffffe321'], dtype=object)

In [90]:
#hist_ded_ids = unique_arr[0:10000]
#len(hist_ded_ids)

10000

In [32]:
#customers_pd_df = customers_df.toPandas()

In [46]:
#hist_ded_ids.count()

device_id    10000
dtype: int64

In [95]:
#customers_pd_df = customers_pd_df.drop(["devide_id"])
#customers_pd_df["device_id"] = hist_ded_ids

In [98]:
#customers_pd_df = customers_pd_df.drop("devide_id", axis=1)

In [100]:
#final_spark_df_customers = spark.createDataFrame(customers_pd_df)

In [101]:
#final_spark_df_customers.write.mode("overwrite").csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data', header=True)

In [102]:
#verify_df = spark.read.csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data', header=True)

In [103]:
#verify_df.show()

+-----------+--------------+--------------------+------+--------------------+--------------------+----------+--------------------+---------+
|customer_id|      username|                name|gender|               email|          occupation| birthdate|             address|device_id|
+-----------+--------------+--------------------+------+--------------------+--------------------+----------+--------------------+---------+
|          1|       robin48|       Jesse Spencer|     M|   udalton@yahoo.com|Pharmacist, commu...|1975-09-24|10305 Scott River...| 000032d7|
|          2|cynthiajackson|     Savannah Daniel|     F|walkerchristopher...|        Set designer|1934-09-28|70884 Andrew Plaz...| 000070cc|
|          3|       ydurham|     Alexander Davis|     M|annlindsey@yahoo.com|Plant breeder/gen...|1975-11-09|0365 Carrie Point...| 00038618|
|          4| murphymichael|      Patrick Cortez|     M| freeves@hotmail.com|Scientist, audiol...|1911-01-20|9864 Brian Walk S...| 000727b1|
|          5|

In [104]:
#hist_ded_ids = ct_hist_df.select("device_id").sample(.01).toPandas()

In [105]:
#unique_arr2 = np.unique(hist_ded_ids.device_id)

In [107]:
#sample_device_ids = [i for i in unique_arr2 if i not in unique_arr]

In [108]:
#len(sample_device_ids)

29261

In [113]:
#array_df = pd.DataFrame(sample_device_ids, columns=['device_id'])
#device_id_spark_df = spark.createDataFrame(array_df)

In [114]:
#device_id_spark_df.write.csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data/device_ids', header=True)

In [20]:
def make_batch_df(max_current_cust_id):
    
    batch_size = random.randint(100, 1000)
    appends = []
    
    for i in range(batch_size):
        appends.append(fake.profile())
        
    appends_df = pd.DataFrame(appends)[["username", "name", "sex", "mail", "job", "birthdate", "address"]]
    appends_df['customer_id'] = list(range(max_current_cust_id, max_current_cust_id+batch_size))
    
    appends_df.rename({'sex': 'gender', 'job': 'position', 'mail':'email'}, axis=1)
    
    return appends_df

In [21]:
new_batch_pd_df = make_batch_df(max_current_cust_id)
new_batch_pd_df_count = new_batch_pd_df.count()

In [22]:
new_batch_spark_df = spark.createDataFrame(new_batch_pd_df)

In [23]:
new_batch_len = new_batch_pd_df_count[0].item()

In [24]:
in_device_id_spark_df = spark.read.csv(os.environ["STORAGE"]+'/cde-workshop/clickthrough/customers/data/device_ids', header=True)

In [25]:
in_device_slice = in_device_id_spark_df.limit(new_batch_len)

In [26]:
from pyspark.sql.functions import monotonically_increasing_id

In [27]:
in_device_slice = in_device_slice.withColumn("mono_id",monotonically_increasing_id() )

In [28]:
new_batch_spark_df = new_batch_spark_df.withColumn("mono_id",monotonically_increasing_id() )

In [29]:
new_batch_spark_append_df = new_batch_spark_df.join(in_device_slice, 'mono_id').drop('mono_id')

In [30]:
new_batch_spark_append_df.dtypes

[('username', 'string'),
 ('name', 'string'),
 ('sex', 'string'),
 ('mail', 'string'),
 ('job', 'string'),
 ('birthdate', 'date'),
 ('address', 'string'),
 ('customer_id', 'bigint'),
 ('device_id', 'string')]

In [31]:
customers_df.dtypes

[('customer_id', 'string'),
 ('username', 'string'),
 ('name', 'string'),
 ('gender', 'string'),
 ('email', 'string'),
 ('occupation', 'string'),
 ('birthdate', 'string'),
 ('address', 'string'),
 ('device_id', 'string')]

In [38]:
customers_df.show()

[Stage 5:>                                                          (0 + 1) / 1]

+-----------+--------------+--------------------+------+--------------------+--------------------+----------+--------------------+---------+
|customer_id|      username|                name|gender|               email|          occupation| birthdate|             address|device_id|
+-----------+--------------+--------------------+------+--------------------+--------------------+----------+--------------------+---------+
|          1|       robin48|       Jesse Spencer|     M|   udalton@yahoo.com|Pharmacist, commu...|1975-09-24|10305 Scott River...| 000032d7|
|          2|cynthiajackson|     Savannah Daniel|     F|walkerchristopher...|        Set designer|1934-09-28|70884 Andrew Plaz...| 000070cc|
|          3|       ydurham|     Alexander Davis|     M|annlindsey@yahoo.com|Plant breeder/gen...|1975-11-09|0365 Carrie Point...| 00038618|
|          4| murphymichael|      Patrick Cortez|     M| freeves@hotmail.com|Scientist, audiol...|1911-01-20|9864 Brian Walk S...| 000727b1|
|          5|

In [ ]:
new_batch_spark_append_df.write.format('parquet').mode("overwrite").saveAsTable('default.newbatch')

In [44]:
new_batch_spark_append_df.createOrReplaceTempView("newbatch")

In [42]:
customers_df.write.mode("overwrite").saveAsTable('default.historical_customers')

In [45]:
spark.sql("SELECT * FROM default.historical_customers LIMIT 10").show()

[Stage 7:>                                                          (0 + 1) / 1]

+-----------+--------------+------------------+------+--------------------+--------------------+----------+--------------------+---------+
|customer_id|      username|              name|gender|               email|          occupation| birthdate|             address|device_id|
+-----------+--------------+------------------+------+--------------------+--------------------+----------+--------------------+---------+
|          1|       robin48|     Jesse Spencer|     M|   udalton@yahoo.com|Pharmacist, commu...|1975-09-24|10305 Scott River...| 000032d7|
|          2|cynthiajackson|   Savannah Daniel|     F|walkerchristopher...|        Set designer|1934-09-28|70884 Andrew Plaz...| 000070cc|
|          3|       ydurham|   Alexander Davis|     M|annlindsey@yahoo.com|Plant breeder/gen...|1975-11-09|0365 Carrie Point...| 00038618|
|          4| murphymichael|    Patrick Cortez|     M| freeves@hotmail.com|Scientist, audiol...|1911-01-20|9864 Brian Walk S...| 000727b1|
|          5|    brittany85

In [47]:
spark.sql("SELECT * FROM newbatch LIMIT 10").show()

+--------------+------------------+---+--------------------+--------------------+----------+--------------------+-----------+---------+
|      username|              name|sex|                mail|                 job| birthdate|             address|customer_id|device_id|
+--------------+------------------+---+--------------------+--------------------+----------+--------------------+-----------+---------+
|       wwatson|    Tiffany Graham|  F|  smithamy@yahoo.com| Landscape architect|1979-08-26|507 Brown Row
Sou...|       9999| 7e00e93d|
|        noah24|Michael Montgomery|  M|tjennings@hotmail...| Administrator, arts|1967-03-28|967 Lori Plains S...|      10000| 7e01296d|
|  sanderssusan| Ronald Barnes Jr.|  M|kellypatel@hotmai...|Producer, televis...|1936-03-26|789 Debra Plain
N...|      10001| 7e033a05|
|        wcantu|      Dawn Wilkins|  F|    jponce@gmail.com|       Advice worker|1928-08-30|Unit 3223 Box 646...|      10002| 7e04addc|
|   kylejackson|       Ian Shaffer|  M|  dale18@

In [48]:
spark.sql("INSERT OVERWRITE TABLE default.historical_customers SELECT * FROM newbatch")

DataFrame[]